# Get the Crystal Structures
Get the crystal structures associated with the melting temperature data of [Seko 2014](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.89.054303).

In [1]:
from tqdm import tqdm_notebook as tqdm
from pymatgen.io.vasp import Poscar
from pymatgen.ext.matproj import MPRester
import pandas as pd

## Load in the Melting Temperature Data
We've stored it in `melting_temperature.csv`

In [2]:
data = pd.read_csv('melting_temperature.csv', header=1)
print('Loaded {} entries'.format(len(data)))

Loaded 248 entries


## Get the Crystal Structures at Each Composition
Following the procedure of [Seko 2017](http://link.aps.org/doi/10.1103/PhysRevB.95.144110), we get the lowest energy ICSD compound at each composition

In [3]:
rester = MPRester()

In [4]:
def get_material_id(composition):
    """Given the composition, get the lowest-energy ICSD structure
    
    Args:
        composition (string): Composition to search
    Return:
        (string) materials_id of desired structure
    """
    
    # Get all structures at this composition
    mids = rester.get_materials_ids(composition)
    
    # Get all structures
    strcs = sum([rester.get_data(mid) for mid in mids], [])
    
    # Get only structures from the ICSD
    strcs = [x for x in strcs if len(x['icsd_ids']) > 0]
    
    # Sort by energy
    strcs = sorted(strcs, key=lambda x: x['energy'])
    return strcs[0]['material_id']

In [5]:
data['material_id'] = [get_material_id(x) for x in tqdm(data['composition'])]

A Jupyter Widget

In [7]:
data['structure'] = [rester.get_structure_by_material_id(x) for x in tqdm(data['material_id'])]

A Jupyter Widget

## Save the Data to Disk
Save it as a CSV file with the structure in POSCAR format

In [8]:
data['poscar'] = [Poscar(x).get_string() for x in tqdm(data['structure'])]

A Jupyter Widget

In [9]:
data.drop(['structure'], 'columns').to_csv('melting_temperature_with_structures.csv', index=False)